In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from datetime import datetime
from sympy import O
import undetected_chromedriver as uc

# 將列標題居中顯示
header_styles = [
    {
        'selector': 'thead th',
        'props': [('text-align', 'center')]
    }
]

# 使用 undetected_chromedriver 初始化 Chrome 瀏覽器
options = uc.ChromeOptions()
driver = uc.Chrome(options=options)

# 定義篩選選項的字典
board_mapping = {
    1:'tech_job',
    2:'softwareengineer',
    3:'f2e',
    4:'job',
    5:'parttime',
    6:'job_search',
    7:'gov_employee',
    8:'oversea_job',
    9:'startup',
    10:'freelancer',
    11:'career_skills_training_field',
    12:'job_seeking_noobs',
    13:'ai_tool_lounge'
}

board_chinese_mapping = {
    'tech_job': '科技業',
    'softwareengineer': '軟體工程師',
    'f2e': '前端工程師',
    'job': '工作',
    'parttime': '打工職缺',
    'job_search': '求職',
    'gov_employee': '公職',
    'oversea_job': '海外工作',
    'startup': '創業',
    'freelancer': '自由工作者',
    'career_skills_training_field': '職涯技能修煉場',
    'job_seeking_noobs': '求職新手互助區',
    'ai_tool_lounge': 'AI工具交誼廳'
}

filter_mapping = {
    1: 'created',#最新發佈
    2: 'like',#心情數
    3: 'collection',#收藏數
    4: 'relevance'#相關度
}

# 選擇看板
board_input = int(input("請輸入數字選擇看板:\n"
                        "1.科技業\n"
                        "2.軟體工程師\n"
                        "3.前端工程師\n"
                        "4.工作\n"
                        "5.打工職缺\n"
                        "6.求職\n"
                        "7.公職\n"
                        "8.海外工作\n"
                        "9.創業\n"
                        "10.自由工作者\n"
                        "11.職涯技能修煉場\n"
                        "12.求職新手互助區\n"
                        "13.AI工具交誼廳"))

board_option = board_mapping.get(board_input)

# 搜尋關鍵字
query = input("請輸入搜尋關鍵字:")

# 選擇篩選的選項
filiter_input = int(input("請輸入數字選要篩選的選項:\n1.最新發佈\n2.心情數\n3.收藏數\n4.相關度"))

# 將選項映射到 filter_mapping 中
filiter_option = filter_mapping.get(filiter_input)

# 開啟條件搜尋後的Dcard網站
driver.get(f'https://www.dcard.tw/search/posts?query={query}&forum={board_option}&sort={filiter_option}')

# 設定捲動螢幕的高度
scroll_height = 100

# 輸入使用者想要捲動的次數
num_scrolls = int(input("Enter the number of scrolls: "))

# 使用 set 來存儲已經爬取的文章標題和連結
visited_articles = set()

# 初始化等待元素的 webdriver 實例
wait = WebDriverWait(driver, 30)

# 設定等待新資料載入的時間（秒）
wait_time = 2

# 初始化一個空的 list，用於存放文章資訊
article_data = []

# 定義一個函數，用於格式化文章標題中包含的 em 元素
def format_text_with_em(element):
    em_elements = element.find_elements(By.TAG_NAME, "em")
    for em_element in em_elements:
        driver.execute_script(
            "var span = document.createElement('span'); "
            "span.style.color = 'red'; "
            "span.style.fontWeight = 'bold'; "
            "span.textContent = arguments[0].textContent; "
            "arguments[0].parentNode.replaceChild(span, arguments[0]);",
            em_element
        )
    return element.get_attribute("outerHTML")

# 定義一個函數，用於提取文章的文字資訊
def extract_text_only(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text()

for _ in range(num_scrolls):
    # 捲動螢幕到最底部
    driver.execute_script(f'window.scrollBy(0, {scroll_height});')

    # 等待新資料載入完成
    sleep(wait_time)

    try:
        # 等待文章元素出現
        articles = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))

        # 提取文章的資訊
        for article in articles:
            title_element = article.find_element(By.TAG_NAME, "a")
            title_with_em = format_text_with_em(title_element)
            urls = title_element.get_attribute("href")

            # 如果当前文章的标题和链接已经在 visited_articles 中，跳过当前循环
            if title_with_em in visited_articles and urls in visited_articles:
                continue

            # 将文章的标题和链接添加到 visited_articles 中
            visited_articles.add(title_with_em)
            visited_articles.add(urls)

            subtitle_element = article.find_element(By.CLASS_NAME, "atm_d2_1gzgpud.atm_ks_15vqwwr.m16n7y82")
            subtitle_with_em = format_text_with_em(subtitle_element)

            heart = article.find_element(By.CLASS_NAME, "atm_lk_i2wt44").text
            message = article.find_element(By.CLASS_NAME, "atm_9s_1txwivl.atm_h_1h6ojuz.atm_ll_exct8b>span").text

            time = article.find_element(By.TAG_NAME, "time").get_attribute("datetime")
            timestamp = datetime.fromisoformat(time.replace('Z', '+00:00'))
            formatted_date = timestamp.strftime("%Y-%m-%d %H:%M")
            
            # 導航到URL並抓取內容
            driver.get(urls)
            
            # 添加你的程式碼以從頁面中抓取內容，例如使用BeautifulSoup或其他方法
            # 例如，如果內容在特定的class中，你可以這樣做：
            try:
                content_element = driver.find_element(By.CLASS_NAME, "atm_vv_1btx8ck.atm_w4_1hnarqo.c1ehvwc9")
                content_text = content_element.text
                
            except Exception as e:
                content_text = "無法檢索內容"
                
            # 將文章資訊加入 list
            # article_data.append([title_with_em, subtitle_with_em, formatted_date, heart, message, urls])
            
            # 將文章資訊加入list of dictionaries
            article_data.append({
                "title": title_with_em,
                "subtitle": subtitle_with_em,
                "時間": formatted_date,
                "愛心數量": heart,
                "留言數": message,
                "url": urls,
                "content": content_text
            })
            
            # 等待1秒
            sleep(1)
            
            # 返回上一頁
            driver.back()

    except Exception as e:
        print("錯誤:", e)

# 將list of lists轉為DataFrame
# df = pd.DataFrame(article_data, columns=["title", "subtitle", "時間", "心情數", "留言數", "url"])

# 將list of dictionaries轉為DataFrame
df = pd.DataFrame(article_data)

# 設定樣式
df_styled = df.style.set_table_styles(header_styles)

# 關閉瀏覽器
driver.quit()

# 將 DataFrame 存成 CSV 檔案（僅文字版本）
# df_text_only = pd.DataFrame(article_data, columns=["title", "subtitle", "時間", "心情數", "留言數", "url"])
df_text_only = pd.DataFrame(article_data)

df_text_only["title"] = df_text_only["title"].apply(extract_text_only)
df_text_only["subtitle"] = df_text_only["subtitle"].apply(extract_text_only)
board_chinese_option = board_chinese_mapping.get(board_option,board_option)
df_text_only.to_csv(f"dcard_{board_chinese_option}板-{query}.csv", encoding="utf-8", index=False)

# 顯示 DataFrame
df_styled

錯誤: Message: stale element reference: stale element not found
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00358D03+51395]
	(No symbol) [0x002C5F61]
	(No symbol) [0x0017E13A]
	(No symbol) [0x00189DA9]
	(No symbol) [0x00182DD1]
	(No symbol) [0x001814D5]
	(No symbol) [0x00183E69]
	(No symbol) [0x00183F00]
	(No symbol) [0x001B6008]
	(No symbol) [0x001B63EB]
	(No symbol) [0x001AE001]
	(No symbol) [0x001D3ED4]
	(No symbol) [0x001ADF7E]
	(No symbol) [0x001D4124]
	(No symbol) [0x001EA570]
	(No symbol) [0x001D3C26]
	(No symbol) [0x001AC629]
	(No symbol) [0x001AD40D]
	GetHandleVerifier [0x006D68D3+3712147]
	GetHandleVerifier [0x00715CBA+3971194]
	GetHandleVerifier [0x00710FA8+3951464]
	GetHandleVerifier [0x00409D09+776393]
	(No symbol) [0x002D1734]
	(No symbol) [0x002CC618]
	(No symbol) [0x002CC7C9]
	(No symbol) [0

,title,subtitle,時間,愛心數量,留言數,url,content
0,#發問 請問要怎麼幫AI角色做語音？,你知道怎麼幫AI角色做語音嗎 ... ？因為Character AI,2024-03-18 06:13,0,0,https://www.dcard.tw/f/softwareengineer/p/255005385,你知道怎麼幫AI角色做語音嗎？因為Character AI有新功能，就是語音，可以用別人做好的，又可以用自己做的。 我之所以會問是因為有些男性角色是10～12歲的小男孩，但聲音卻是20歲以上的成人的，覺得很奇怪。所以我想製作比較符合小男孩的聲線。
1,#發問 #新手 SQL 語法 Access 資料庫,丟AI感覺還少了什麼 也是出現錯誤🥲,2024-03-14 16:40,3,7,https://www.dcard.tw/f/softwareengineer/p/254974792,目前想生出這個表 我按照提示把幾個欄inner join起來 之後依照提示使用Order by 來排列 結果跳出錯誤 是還少了什麼參數嗎？ 原題目如下 找不到地方可以問資料庫相關題目 丟AI感覺還少了什麼 也是出現錯誤🥲
2,#發問 請益 韌體轉軟體所需之進階課程,，有沒有推薦的AI或大數據相關,2024-03-01 01:59,5,6,https://www.dcard.tw/f/softwareengineer/p/254833984,目前寫韌體（家電產品+EC）共五年 因為諸多原因，想換換跑道試試 想請問在業界的前輩們，有沒有推薦的AI或大數據相關的python課程 （不教基礎、不教google隨便都查得到的） 可以上呢？ 先謝謝各位～
3,#發問 #新手 #討論 #發問 #職涯 轉職軟體工程師，找台北學習的夥伴，Python學習中，和請教未來發展方向,"端以外的方向(AI, 數據,雲端",2024-02-22 15:03,10,14,https://www.dcard.tw/f/softwareengineer/p/254757049,"小弟在封裝廠做基板設計工程師6年，工作疲勞和覺得上限太低，休息去菲律賓語言學校半年，認識一位後端工程師的日本室友，發現他認真兩個月等於我的年薪🤣 因此備受啟發，開始了轉職之路，但我不知什麼樣的道路合適，先從網路資源豐富的python開始 學習進度: 花1個月上完: 彭彭的python課程 花4個月到現在: Udemy:100 Days of Code: The Complete Python Pro Bootcamp(Dr. Angela Yu) 學到 63days 進度是不是有點緩慢 ? 😅 因為老師的內容包山包海，最後幾章是大型作業，還要自己完能一些小型side project 100 Days of Code: The Complete Python Pro Bootcamp Master Python by building 100 projects in 100 days. Learn data science, automation, build websites, games and apps! www.udemy.com 未來規劃: 在網路上蒐集的學習進程(後端工程師為主)，不知道是否有誤? 3月: 預計完成python 100days 4-5月: Udemy:The Complete 2024 Web Development Bootcamp(Dr. Angela Yu) (想接觸前端三大技能，看自己喜不喜歡) 6月: 網路基礎, 資料庫管理, 版本控制 7月: 後端框架 Django 或者 JavaScript + Node.js 8月: 想找個學習機構: 首選: Appworks School (4,5月會寄資料 沒上等90天再一次) 第二: Alpha Camp 後端班 問題請教: 1. 平常會去大學圖書館看一天，但有時候進度卡住會怠惰，想找人一起學習，互相勉勵😁 2. 各位前輩是如何知道，自己適合哪發展方向 ?? 就我個人而言，就是不斷的接觸學習，看自己在不在行，至於前後端以外的方向(AI, 數據,雲端,APP工程師)，有人分享非本科很難駕馭，所需知識量差距太大，所以想先從進入後端再說。 3. 個人的學習計劃是否有問題? 我一直不知道，當我把課上完，我的程度到哪? 花多久上完合適? (好希望有像RO 之類的技能表，python Lv.10 -> Django(Lv.1/10)，腦動大開中🤪 Job transfer conditions (Backend): python Lv.10/10,Django(#Lv.1/10), #HTTP,(5) #Git(3), ......) 各位前輩有什麼建議，請不吝賜教，小弟第一次發文，有所不周之處還請見諒，想謝看完。"
4,#討論 資工請益,本人初學者，覺得ai很方便,2024-02-22 09:44,17,30,https://www.dcard.tw/f/softwareengineer/p/254753531,本人初學者，覺得ai很方便，目前只會用GPT幫忙寫程式，寫出來的也淺懂而已，想請問 1.如果gpt可以代替人工寫程式，那我還要深學嗎？ 2.如果要深學，刷題是最快的方式嗎？謝謝。 更1- 本人是非本科上來的，每個留言我都看了，謝謝各位的建議。 我會努力的🥹🥹
5,#發問 Junior 策略PM薪資請益,38k 現今工作是偏軟體開發 AI,2024-02-21 13:34,6,33,https://www.dcard.tw/f/softwareengineer/p/254744523,工作經驗約兩年 走過測試38k 現今工作是偏軟體開發 AI chatbot &網頁asp.net core 請問新職缺是junior 策略PM以業界pm開薪的範圍外商49k是否認為會太少嗎？？ *更 謝謝大家的批評指教🙏🏻 除了薪資外，目前職位上的專案進度還在階段中， 考量角度還有是否該將手上的專案完整走完～ 新職缺是主管有意內推去其他部門試看看 我個人是想先去面試，多方思考薪資、職務內容、與目前職務比較是否更有發展等等。
6,#發問 python *.pyd與*.dll一樣嗎？,，問AI也是這樣回答，但我改了,2024-02-15 14:31,9,23,https://www.dcard.tw/f/softwareengineer/p/254683562,"由於我是用python開發程式，要將其轉成.dll檔，使用c++來調用dll裡面的函數，但我找的教學，都是將python程式轉成py格式。 我有搜尋pyd如何轉換成dll，都沒有相關的教學，大多都說pyd與dll是相同的，只要改副檔名就好，問AI也是這樣回答，但我改了之後，用以下的程式碼來測試調用dll，都無法成功 用python調用dll： ``` import ctypes # 加载DLL dll_path = ""./_first.dll"" my_dll = ctypes.CDLL(dll_path) # 調用DLL中的函数 result = my_dll.hello() print(""Result:"", result) ``` 用c++調用dll： ``` #include #include typedef void (*YourFunctionType)(); int main() { // 加载 DLL HINSTANCE hDLL = LoadLibraryA(""_first.dll""); if (hDLL != NULL) { // 获取函数地址 YourFunctionType yourFunction = (YourFunctionType)GetProcAddress(hDLL, ""hello""); if (yourFunction != NULL) { // 调用函数 yourFunction(); } else { std::cerr << ""Failed to get function address."" << std::endl; } // 释放 DLL FreeLibrary(hDLL); } else { std::cerr << ""Failed to load DLL."" << std::endl; } return 0; } ``` 要轉換的python程式碼： ``` def hello(): print(""Hello World!"") ``` 這兩種方法都沒辦法成功調用，都說沒有hello()這個函數，但我使用python調用pyd，是可以正常的調用hello()。 所以想詢問一下，有甚麼方法可以解決這問題？ PS：在官方文件也有說明相關的問題( Python在Windows上的常见问题 — Python 3.6.15 文档 https://docs.python.org/zh-cn/3.6/faq/windows.html#is-a-pyd-file-the-same-as-a-dll)%EF%BC%8C%E4%BD%86%E6%88%91%E6%9C%89%E7%9C%8B%E6%B2%92%E6%9C%89%E6%87%82.... docs.python.org"
